# Sequential Multi-Task IQA Training Pipeline

This notebook trains three tasks sequentially:
1. **Stage 1**: Scene Classification
2. **Stage 2**: Distortion Classification (building on Scene knowledge)
3. **Stage 3**: Quality Assessment (building on Scene + Distortion knowledge)

Each stage is in a separate cell, so if one fails, you can fix it and continue from that stage.

## Configuration

In [ ]:
# Configuration parameters
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Training configuration
DATASET_PATHS = ["datasets/koniq-10k/"]  # Change to your dataset
OUTPUT_DIR = "outputs/10281300"
BASE_MODEL = "src/owl3"

# Training hyperparameters
MAX_STEPS = -1  # Number of steps per stage (-1 for full epochs)
NUM_TRAIN_EPOCHS = 3
BATCH_SIZE = 1
GRAD_ACCUM = 8
LEARNING_RATE = 2e-4
EVAL_STEPS = 100
SAVE_STEPS = 100
LOGGING_STEPS = 50

# Early stopping configuration
EARLY_STOPPING_PATIENCE = 5  # Stop if no improvement after 5 evaluations

# LoRA parameters
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05

# Loss weights
USE_FIDELITY_LOSS = True

print("✅ Configuration set!")
print(f"📁 Dataset: {DATASET_PATHS}")
print(f"📁 Output: {OUTPUT_DIR}")
print(f"🎯 Training: {NUM_TRAIN_EPOCHS} epochs, max {MAX_STEPS} steps per stage")
print(f"⚙️  Batch Size: {BATCH_SIZE} × {GRAD_ACCUM} = {BATCH_SIZE * GRAD_ACCUM}")
print(f"🛑 Early Stopping: patience={EARLY_STOPPING_PATIENCE} (stop if no improvement after {EARLY_STOPPING_PATIENCE} evaluations)")

✅ Configuration set!
📁 Dataset: ['datasets/koniq-10k/']
📁 Output: outputs/10270900
🎯 Training: 3 epochs, max -1 steps per stage
⚙️  Batch Size: 1 × 8 = 8
🛑 Early Stopping: patience=5 (stop if no improvement after 5 evaluations)


## Imports and Setup

In [5]:
import sys
from pathlib import Path
import torch

from transformers import (
    AutoTokenizer,
    TrainingArguments,
    set_seed,
)

# Add src to path
sys.path.insert(0, str(Path.cwd()))

from src.new_train.model_wrapper import IQAModelWrapper
from src.new_train.dataset_adapter import IQAPairDataset, collate_fn_pair
from src.new_train.processor_no_cut import create_processor_no_cut
from src.new_train.iqa_trainer import IQATrainer
from src.new_train.plot_utils import plot_training_curves

# Import collate functions
from src.new_train.train_scene import collate_fn_scene
from src.new_train.train_distortion import collate_fn_distortion

# Set seed
set_seed(42)

print("✅ Imports completed!")

✅ Imports completed!


## Initialize Model (Run Once)

In [6]:
# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print("🔧 Loading tokenizer and processor...")
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True,
)
processor = create_processor_no_cut(tokenizer)

print("🔧 Initializing model with LoRA...")
model = IQAModelWrapper(
    model_name_or_path=BASE_MODEL,
    lora_r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    weight_fidelity=1.0 if USE_FIDELITY_LOSS else 0.0,
)

print("\n✅ Model initialized!")
print(f"📊 Model will be trained sequentially on 3 tasks")

🔧 Loading tokenizer and processor...
🔧 Initializing model with LoRA...
use flash_attn rotary


HyperQwen2ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


trainable params: 43,356,160 || all params: 8,115,903,040 || trainable%: 0.5342

✅ Model initialized!
📊 Model will be trained sequentially on 3 tasks


---
## Stage 1: Scene Classification Training

Train the model to classify scene types (e.g., landscape, cityscape, human, etc.)

In [7]:
print("="*80)
print("STAGE 1/3: Scene Classification Training")
print("="*80)

# Create dataset
print("\n📊 Creating scene classification dataset...")
dataset_paths = [Path(p) for p in DATASET_PATHS]
train_dataset_scene = IQAPairDataset(
    dataset_paths=dataset_paths,
    processor=processor,
    tokenizer=tokenizer,
    split="training",
    use_scene_labels=True,
    use_distortion_labels=False,
)

val_dataset_scene = IQAPairDataset(
    dataset_paths=dataset_paths,
    processor=processor,
    tokenizer=tokenizer,
    split="validation",
    use_scene_labels=True,
    use_distortion_labels=False,
)

print(f"✅ Training dataset size: {len(train_dataset_scene)}")
print(f"✅ Validation dataset size: {len(val_dataset_scene)}")

STAGE 1/3: Scene Classification Training

📊 Creating scene classification dataset...
✅ Training dataset size: 7252
✅ Validation dataset size: 1813


In [8]:
# Training arguments for Scene task
output_dir_scene = f"{OUTPUT_DIR}/01_scene"
training_args_scene = TrainingArguments(
    output_dir=output_dir_scene,
    num_train_epochs=NUM_TRAIN_EPOCHS if MAX_STEPS <= 0 else 1,
    max_steps=MAX_STEPS if MAX_STEPS > 0 else -1,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.0,
    logging_steps=LOGGING_STEPS,
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    save_strategy="steps",
    save_steps=SAVE_STEPS,
    save_total_limit=2,
    bf16=True,
    dataloader_num_workers=12,
    remove_unused_columns=False,
    report_to="none",
    load_best_model_at_end=True,  # Load best model based on eval_loss
    metric_for_best_model="eval_loss",
    greater_is_better=False,  # Lower loss is better
)

print("✅ Training arguments configured for Scene task")
print("   📌 Will load best model (lowest eval_loss) at end")
print(f"   📌 Early stopping: patience={EARLY_STOPPING_PATIENCE}")

✅ Training arguments configured for Scene task
   📌 Will load best model (lowest eval_loss) at end
   📌 Early stopping: patience=5


In [9]:
# Custom trainer for scene task
from transformers import EarlyStoppingCallback

class SceneTrainer(IQATrainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs = model.forward_scene_task(
            pixel_values_A=inputs["pixel_values_A"],
            input_ids_scene_A=inputs["input_ids_scene_A"],
            attention_mask_scene_A=inputs["attention_mask_scene_A"],
            labels_scene_A=inputs["labels_scene_A"],
            media_offset_A=inputs["media_offset_A"],
            pixel_values_B=inputs["pixel_values_B"],
            input_ids_scene_B=inputs["input_ids_scene_B"],
            attention_mask_scene_B=inputs["attention_mask_scene_B"],
            labels_scene_B=inputs["labels_scene_B"],
            media_offset_B=inputs["media_offset_B"],
        )
        loss = outputs["loss"]
        return (loss, outputs) if return_outputs else loss
    
    def prediction_step(self, model, inputs, prediction_loss_only: bool, ignore_keys=None):
        has_labels = "labels_scene_A" in inputs and "labels_scene_B" in inputs
        with torch.no_grad():
            if has_labels:
                loss, outputs = self.compute_loss(model, inputs, return_outputs=True)
                loss = loss.mean().detach()
            else:
                loss = None
        return (loss, None, None)

# Create early stopping callback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=EARLY_STOPPING_PATIENCE,
    early_stopping_threshold=0.0,  # Any improvement counts
)

# Create trainer
trainer_scene = SceneTrainer(
    model=model,
    args=training_args_scene,
    train_dataset=train_dataset_scene,
    eval_dataset=val_dataset_scene,
    data_collator=collate_fn_scene,
    callbacks=[early_stopping_callback],
)

print("✅ Scene trainer created!")
print(f"   🛑 Early stopping enabled: patience={EARLY_STOPPING_PATIENCE}")

✅ Scene trainer created!
   🛑 Early stopping enabled: patience=5


In [10]:
# Train Scene task
print("\n🚀 Starting Scene classification training...")
print("="*80)
trainer_scene.train()
print("="*80)
print("\n✅ Scene training completed!")

# Generate plots
print("\n📊 Generating training curves...")
plot_training_curves(output_dir=output_dir_scene)
print(f"✅ Plots saved to {output_dir_scene}/")


🚀 Starting Scene classification training...


The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.46 `position_ids` will be removed and `position_embeddings` will be mandatory.


Step,Training Loss,Validation Loss
100,0.053800,0.041473
200,0.035700,0.035143
300,0.042500,0.032507
400,0.032900,0.032796
500,0.023700,0.045602
600,0.045000,0.035222
700,0.063000,0.037553
800,0.031900,0.038823


Epoch 0.06 | Loss: 0.3504
Epoch 0.11 | Loss: 0.0538
Epoch 0.11 | Loss: 0.0538

[DEBUG] Collected 0 predictions

📊 Validation Results at Epoch 0.11
  Loss:       0.041473


[DEBUG] Collected 0 predictions

📊 Validation Results at Epoch 0.11
  Loss:       0.041473

Epoch 0.17 | Loss: 0.0402
Epoch 0.17 | Loss: 0.0402
Epoch 0.22 | Loss: 0.0357
Epoch 0.22 | Loss: 0.0357

[DEBUG] Collected 0 predictions

📊 Validation Results at Epoch 0.22
  Loss:       0.035143


[DEBUG] Collected 0 predictions

📊 Validation Results at Epoch 0.22
  Loss:       0.035143

Epoch 0.28 | Loss: 0.0391
Epoch 0.28 | Loss: 0.0391
Epoch 0.33 | Loss: 0.0425
Epoch 0.33 | Loss: 0.0425

[DEBUG] Collected 0 predictions

📊 Validation Results at Epoch 0.33
  Loss:       0.032507


[DEBUG] Collected 0 predictions

📊 Validation Results at Epoch 0.33
  Loss:       0.032507

Epoch 0.39 | Loss: 0.0324
Epoch 0.39 | Loss: 0.0324
Epoch 0.44 | Loss: 0.0329
Epoch 0.44 | Loss: 0.0329

[DEBUG] Collected 0 predictions

📊 Validation Resul

---
## Stage 2: Distortion Classification Training

Build on Scene knowledge to classify distortion types (e.g., blur, noise, compression, etc.)

In [11]:
print("="*80)
print("STAGE 2/3: Distortion Classification Training")
print("="*80)
print("Building on Scene classification knowledge...")

# Create dataset
print("\n📊 Creating distortion classification dataset...")
train_dataset_distortion = IQAPairDataset(
    dataset_paths=dataset_paths,
    processor=processor,
    tokenizer=tokenizer,
    split="training",
    use_scene_labels=False,
    use_distortion_labels=True,
)

val_dataset_distortion = IQAPairDataset(
    dataset_paths=dataset_paths,
    processor=processor,
    tokenizer=tokenizer,
    split="validation",
    use_scene_labels=False,
    use_distortion_labels=True,
)

print(f"✅ Training dataset size: {len(train_dataset_distortion)}")
print(f"✅ Validation dataset size: {len(val_dataset_distortion)}")

STAGE 2/3: Distortion Classification Training
Building on Scene classification knowledge...

📊 Creating distortion classification dataset...
✅ Training dataset size: 7252
✅ Validation dataset size: 1813


In [12]:
# Training arguments for Distortion task
output_dir_distortion = f"{OUTPUT_DIR}/02_distortion"
training_args_distortion = TrainingArguments(
    output_dir=output_dir_distortion,
    num_train_epochs=NUM_TRAIN_EPOCHS if MAX_STEPS <= 0 else 1,
    max_steps=MAX_STEPS if MAX_STEPS > 0 else -1,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.0,
    logging_steps=LOGGING_STEPS,
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    save_strategy="steps",
    save_steps=SAVE_STEPS,
    save_total_limit=2,
    bf16=True,
    dataloader_num_workers=12,
    remove_unused_columns=False,
    report_to="none",
    load_best_model_at_end=True,  # Load best model based on eval_loss
    metric_for_best_model="eval_loss",
    greater_is_better=False,  # Lower loss is better
)

print("✅ Training arguments configured for Distortion task")
print("   📌 Will load best model (lowest eval_loss) at end")
print(f"   📌 Early stopping: patience={EARLY_STOPPING_PATIENCE}")

✅ Training arguments configured for Distortion task
   📌 Will load best model (lowest eval_loss) at end
   📌 Early stopping: patience=5


In [13]:
# Custom trainer for distortion task
class DistortionTrainer(IQATrainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs = model.forward_distortion_task(
            pixel_values_A=inputs["pixel_values_A"],
            input_ids_distortion_A=inputs["input_ids_distortion_A"],
            attention_mask_distortion_A=inputs["attention_mask_distortion_A"],
            labels_distortion_A=inputs["labels_distortion_A"],
            media_offset_A=inputs["media_offset_A"],
            pixel_values_B=inputs["pixel_values_B"],
            input_ids_distortion_B=inputs["input_ids_distortion_B"],
            attention_mask_distortion_B=inputs["attention_mask_distortion_B"],
            labels_distortion_B=inputs["labels_distortion_B"],
            media_offset_B=inputs["media_offset_B"],
        )
        loss = outputs["loss"]
        return (loss, outputs) if return_outputs else loss
    
    def prediction_step(self, model, inputs, prediction_loss_only: bool, ignore_keys=None):
        has_labels = "labels_distortion_A" in inputs and "labels_distortion_B" in inputs
        with torch.no_grad():
            if has_labels:
                loss, outputs = self.compute_loss(model, inputs, return_outputs=True)
                loss = loss.mean().detach()
            else:
                loss = None
        return (loss, None, None)

print("✅ DistortionTrainer class defined!")

✅ DistortionTrainer class defined!


In [14]:
# Create distortion trainer
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=EARLY_STOPPING_PATIENCE,
    early_stopping_threshold=0.0,
)

trainer_distortion = DistortionTrainer(
    model=model,
    args=training_args_distortion,
    train_dataset=train_dataset_distortion,
    eval_dataset=val_dataset_distortion,
    data_collator=collate_fn_distortion,
    callbacks=[early_stopping_callback],
)

print("✅ Distortion trainer created with early stopping!")

✅ Distortion trainer created with early stopping!


In [15]:
# Train Distortion task
print("\n🚀 Starting Distortion classification training...")
print("="*80)
trainer_distortion.train()
print("="*80)
print("\n✅ Distortion training completed!")

# Generate plots
print("\n📊 Generating training curves...")
plot_training_curves(output_dir=output_dir_distortion)
print(f"✅ Plots saved to {output_dir_distortion}/")


🚀 Starting Distortion classification training...


Step,Training Loss,Validation Loss
100,0.065400,0.053710
200,0.043300,0.059190
300,0.044100,0.050715
400,0.045500,0.045591
500,0.040100,0.047934
600,0.040600,0.049053
700,0.033900,0.050499
800,0.036900,0.044182
900,0.036000,0.051400
1000,0.026400,0.064265


Epoch 0.06 | Loss: 0.1391
Epoch 0.11 | Loss: 0.0654
Epoch 0.11 | Loss: 0.0654

[DEBUG] Collected 0 predictions

📊 Validation Results at Epoch 0.11
  Loss:       0.053710


[DEBUG] Collected 0 predictions

📊 Validation Results at Epoch 0.11
  Loss:       0.053710

Epoch 0.17 | Loss: 0.0456
Epoch 0.17 | Loss: 0.0456
Epoch 0.22 | Loss: 0.0433
Epoch 0.22 | Loss: 0.0433

[DEBUG] Collected 0 predictions

📊 Validation Results at Epoch 0.22
  Loss:       0.059190


[DEBUG] Collected 0 predictions

📊 Validation Results at Epoch 0.22
  Loss:       0.059190

Epoch 0.28 | Loss: 0.0445
Epoch 0.28 | Loss: 0.0445
Epoch 0.33 | Loss: 0.0441
Epoch 0.33 | Loss: 0.0441

[DEBUG] Collected 0 predictions

📊 Validation Results at Epoch 0.33
  Loss:       0.050715


[DEBUG] Collected 0 predictions

📊 Validation Results at Epoch 0.33
  Loss:       0.050715

Epoch 0.39 | Loss: 0.0504
Epoch 0.39 | Loss: 0.0504
Epoch 0.44 | Loss: 0.0455
Epoch 0.44 | Loss: 0.0455

[DEBUG] Collected 0 predictions

📊 Validation Resul

---
## Stage 3: Quality Assessment Training

Build on Scene + Distortion knowledge to predict image quality scores

In [16]:
print("="*80)
print("STAGE 3/3: Quality Assessment Training")
print("="*80)
print("Building on Scene + Distortion classification knowledge...")

# Create dataset
print("\n📊 Creating quality assessment dataset...")
train_dataset_quality = IQAPairDataset(
    dataset_paths=dataset_paths,
    processor=processor,
    tokenizer=tokenizer,
    split="training",
    use_scene_labels=False,
    use_distortion_labels=False,
)

val_dataset_quality = IQAPairDataset(
    dataset_paths=dataset_paths,
    processor=processor,
    tokenizer=tokenizer,
    split="validation",
    use_scene_labels=False,
    use_distortion_labels=False,
)

print(f"✅ Training dataset size: {len(train_dataset_quality)}")
print(f"✅ Validation dataset size: {len(val_dataset_quality)}")

STAGE 3/3: Quality Assessment Training
Building on Scene + Distortion classification knowledge...

📊 Creating quality assessment dataset...
✅ Training dataset size: 7252
✅ Validation dataset size: 1813


In [17]:
# Training arguments for Quality task
output_dir_quality = f"{OUTPUT_DIR}/03_quality"
training_args_quality = TrainingArguments(
    output_dir=output_dir_quality,
    num_train_epochs=NUM_TRAIN_EPOCHS if MAX_STEPS <= 0 else 1,
    max_steps=MAX_STEPS if MAX_STEPS > 0 else -1,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.0,
    logging_steps=LOGGING_STEPS,
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    save_strategy="steps",
    save_steps=SAVE_STEPS,
    save_total_limit=2,
    bf16=True,
    dataloader_num_workers=12,
    remove_unused_columns=False,
    report_to="none",
    load_best_model_at_end=True,  # Load best model based on eval_plcc
    metric_for_best_model="eval_plcc",
    greater_is_better=True,  # Higher PLCC is better
)

print("✅ Training arguments configured for Quality task")
print("   📌 Will load best model (highest eval_plcc) at end")

✅ Training arguments configured for Quality task
   📌 Will load best model (highest eval_plcc) at end


In [18]:
# Create trainer for quality task (uses standard IQATrainer)
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=EARLY_STOPPING_PATIENCE,
    early_stopping_threshold=0.0,
)

trainer_quality = IQATrainer(
    model=model,
    args=training_args_quality,
    train_dataset=train_dataset_quality,
    eval_dataset=val_dataset_quality,
    data_collator=collate_fn_pair,
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback],
)

print("✅ Quality trainer created with early stopping!")

✅ Quality trainer created with early stopping!


/home/wytsai7660/code/iqa-project/src/new_train/iqa_trainer.py:68: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `IQATrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


In [19]:
# Train Quality task
print("\n🚀 Starting Quality assessment training...")
print("="*80)
trainer_quality.train()
print("="*80)
print("\n✅ Quality training completed!")

# Generate plots
print("\n📊 Generating training curves...")
try:
    from src.new_train.plot_utils import plot_metrics_summary, plot_correlation_metrics
    plot_training_curves(output_dir_quality)
    plot_metrics_summary(output_dir_quality)
    plot_correlation_metrics(output_dir_quality)
    print("✅ All plots saved!")
except Exception as e:
    print(f"⚠️  Could not generate all plots: {e}")
    plot_training_curves(output_dir_quality)

print(f"✅ Plots saved to {output_dir_quality}/")


🚀 Starting Quality assessment training...


Step,Training Loss,Validation Loss,Mae,Mse,Rmse,Plcc,Srcc
100,12450.152500,nan,0.358433,0.170510,0.412929,0.941295,0.930138
200,953148.720000,nan,0.192019,0.060161,0.245277,0.944881,0.933592
300,1118878.080000,nan,0.298067,0.127900,0.357631,0.934818,0.928414
400,242715340.800000,nan,0.206402,0.066827,0.258510,0.947093,0.940462
500,115057295.360000,nan,0.202155,0.065722,0.256364,0.949322,0.938725
600,4889177.600000,nan,0.256306,0.094954,0.308146,0.945342,0.940845
700,9975.336900,nan,0.243136,0.088088,0.296796,0.952253,0.945019
800,143704.730000,nan,0.173627,0.049772,0.223096,0.954000,0.944444
900,1614763.680000,nan,0.169912,0.048753,0.220801,0.954288,0.945611
1000,4607913.920000,nan,0.205648,0.063888,0.252761,0.954663,0.947916


Epoch 0.06 | Loss: 8625001594.8800
Epoch 0.11 | Loss: 12450.1525
Epoch 0.11 | Loss: 12450.1525

[DEBUG] Collected 1813 predictions
[DEBUG] Computing metrics: pred shape=(1813,), gt shape=(1813,)
[DEBUG] Computed metrics: {'mae': 0.3584327139515148, 'mse': 0.170510122627504, 'rmse': 0.4129287137358021, 'plcc': 0.9412946800950742, 'srcc': 0.9301384944331513}
[DEBUG] Added metrics to output.metrics: ['eval_mae', 'eval_mse', 'eval_rmse', 'eval_plcc', 'eval_srcc']

📊 Validation Results at Epoch 0.11
  Loss:       nan
  MAE:        0.3584
  RMSE:       0.4129
  ------------------------------------------------------------------
  Correlation Metrics:
  PLCC:       0.9413  [██████████████████░░]
  SRCC:       0.9301  [██████████████████░░]


[DEBUG] Collected 1813 predictions
[DEBUG] Computing metrics: pred shape=(1813,), gt shape=(1813,)
[DEBUG] Computed metrics: {'mae': 0.3584327139515148, 'mse': 0.170510122627504, 'rmse': 0.4129287137358021, 'plcc': 0.9412946800950742, 'srcc': 0.93013849443

---
## Save Final Model

In [20]:
print("="*80)
print("SAVING FINAL MODEL")
print("="*80)

final_path = f"{OUTPUT_DIR}/final_model"
model.model.save_pretrained(final_path)
tokenizer.save_pretrained(final_path)

print(f"✅ Final model saved to: {final_path}")
print("\n" + "="*80)
print("🎉 SEQUENTIAL TRAINING PIPELINE COMPLETED!")
print("="*80)
print(f"\n📊 Results:")
print(f"  Stage 1 (Scene):      {output_dir_scene}/")
print(f"  Stage 2 (Distortion): {output_dir_distortion}/")
print(f"  Stage 3 (Quality):    {output_dir_quality}/")
print(f"  Final Model:          {final_path}/")
print()

SAVING FINAL MODEL
✅ Final model saved to: outputs/10270900/final_model

🎉 SEQUENTIAL TRAINING PIPELINE COMPLETED!

📊 Results:
  Stage 1 (Scene):      outputs/10270900/01_scene/
  Stage 2 (Distortion): outputs/10270900/02_distortion/
  Stage 3 (Quality):    outputs/10270900/03_quality/
  Final Model:          outputs/10270900/final_model/

✅ Final model saved to: outputs/10270900/final_model

🎉 SEQUENTIAL TRAINING PIPELINE COMPLETED!

📊 Results:
  Stage 1 (Scene):      outputs/10270900/01_scene/
  Stage 2 (Distortion): outputs/10270900/02_distortion/
  Stage 3 (Quality):    outputs/10270900/03_quality/
  Final Model:          outputs/10270900/final_model/



---
## Evaluation (Optional)

Evaluate the final model on test set

In [ ]:
# You can evaluate on test set here if needed
# Example:
# test_dataset = IQAPairDataset(
#     dataset_paths=dataset_paths,
#     processor=processor,
#     tokenizer=tokenizer,
#     split="testing",
# )
# 
# test_results = trainer_quality.evaluate(test_dataset)
# print(test_results)

print("💡 To evaluate the model, use the eval_sequential_model.py script:")
print(f"   python eval_sequential_model.py --model_path {final_path} --dataset_paths {' '.join(DATASET_PATHS)} --split testing")

💡 To evaluate the model, use the eval_sequential_model.py script:
   python eval_sequential_model.py --model_path outputs/10270900/final_model --dataset_paths datasets/koniq-10k/ --split testing


: 